In [5]:
# Dependencies
import numpy as np
import tweepy
import time
import json
import pandas as pd
import queue
from requests_toolbelt.threaded import pool

# Twitter API Keys
# consumer_key = "TEs2QmTsz4ylhWSqXWHOvVn99"
# consumer_secret = "yA5M95uhTdeAKLondYQ9LDMPR5LWKc6wX70Yry1oRDHWtBN7gg"
# access_token = "400199431-wb53H1jfbkDwY0xVS9aZbgWZrWlXcWkhXdFuBKMh"
# access_token_secret = "IRvQVov2vmopxjeTutR0jn57lnMlYtr8syHQrGbLgbmW9"

consumer_key = "PAmpYOq5LwscMnRSFBQqwNjxH"
consumer_secret = "uvvEfLcCoU9zBsRwc3wbWNGwUy7BUVtYH3376MHpLDy7oZ6qUK"
access_token = "907734237478080523-BAtPF5z8EzzwyPef0f04JC12LOQSCim"
access_token_secret = "0F5xIDmAwVAy7pQcOWs0mMFDedr9po1tLJuLNk8r3gMuG"

# Twitter Credentials
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), wait_on_rate_limit = True, wait_on_rate_limit_notify = True)


In [6]:
# Get a list of locations in and outside the belt (in-belt, out-belt)
base_data = pd.read_csv("Zipcode_States_With_Geo_Cordinates.csv")
lat = base_data.groupby("State")["Latitude"].mean()
lon = base_data.groupby("State")["Longitude"].mean()
dfff = {"lat":[], "lon":[]}
for each in lat.index:
    dfff["lat"].append(lat[each])
    dfff["lon"].append(lon[each])
hah = pd.DataFrame(dfff)
base_data_state = hah.set_index([lat.index])

base_data_state.head(10)

,lat,lon
State,,
Alabama,32.886361,-86.813639
Alaska,61.456423,-152.486981
Arizona,33.610082,-111.647927
Arkansas,35.124723,-92.402676
California,36.170738,-119.792701
Colorado,39.264176,-105.317544
Connecticut,41.563130,-72.804928
Delaware,39.300372,-75.526776
District of Columbia,38.896231,-77.021235


In [7]:
base_data_state = base_data_state.reset_index()
base_data_state

,State,lat,lon
0,Alabama,32.886361,-86.813639
1,Alaska,61.456423,-152.486981
2,Arizona,33.610082,-111.647927
3,Arkansas,35.124723,-92.402676
4,California,36.170738,-119.792701
5,Colorado,39.264176,-105.317544
6,Connecticut,41.563130,-72.804928
7,Delaware,39.300372,-75.526776
8,District of Columbia,38.896231,-77.021235
9,Florida,28.129716,-81.956165


In [8]:
tweet_df = {}
tweet_df = pd.DataFrame(tweet_df)
tweet_df["state"] = ""
tweet_df["text"] = ""
tweet_df["lat"] = ""
tweet_df["lon"] = ""
tweet_df["followers"] = ""
tweet_df["favorites"] = ""
tweet_df["friends"] = ""
tweet_df["statuses_count"] = ""
tweet_df["compound"] = ""
tweet_df["pos"] = ""
tweet_df["neg"] = ""


tweet_df.head()


,state,text,lat,lon,followers,favorites,friends,statuses_count,compound,pos,neg


In [9]:
# Loop through DataFrame for each Latitude and Longitude
k = 0
for index, row in base_data_state.iterrows():    
    
    # Get Tweets for all locations
    location = "%s,%s,100mi" % (row["lat"], row["lon"])
    p = pool.Pool.from_urls(api.search("", geocode = location, count=100), num_processes = 100)
    p.join_all()
#     public_tweets = api.search("", geocode = location, count=100)
    
    # Extract all hashtags from tweets directed to the account
#     for tweet in public_tweets["statuses"]:

#         # Print tweet in JSON
#         # print(json.dumps(tweet, sort_keys=True, indent=4, separators=(',', ': ')))

#         # Print the Tweet Text         
#         print(tweet["text"])

#         # Print tweet text
#         tweet_df.set_value(k, "state", row["State"])
#         tweet_df.set_value(k, "text", tweet["text"])
#         tweet_df.set_value(k, "lat", row["lat"])
#         tweet_df.set_value(k, "lon", row["lon"])

#         tweet_df.set_value(k, "followers", tweet["user"]["followers_count"])
#         tweet_df.set_value(k, "favorites", tweet["user"]["favourites_count"])
#         tweet_df.set_value(k, "friends", tweet["user"]["friends_count"])
#         tweet_df.set_value(k, "statuses_count", tweet["user"]["statuses_count"])

#         # Save to a csv for every state
#         tweet_df.to_csv("Tweet_Dump3.csv")
        
#         k = k + 1
         

Exception in thread 7efee40b-ef3e-4fc9-92ec-2edcf84dadbb:
Traceback (most recent call last):
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests_toolbelt/threaded/thread.py", line 30, in _handle_request
    response = self._session.request(**kwargs)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/sessions.py", line 504, in request
    prep = self.prepare_request(req)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/sessions.py", line 436, in prepare_request
    hooks=merge_hooks(request.hooks, self.hooks),
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/models.py", line 302, in prepare
    self.prepare_url(url, params)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/models.py", line 382, in prepare_url
    raise MissingSchema(error)
requests.exceptions.MissingSchema: Invalid URL 'statuses': No schema supplied. Perhaps you meant http:

Exception in thread f99f2b32-27fb-4e05-8da1-47051d08ab42:
Traceback (most recent call last):
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests_toolbelt/threaded/thread.py", line 30, in _handle_request
    response = self._session.request(**kwargs)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/sessions.py", line 504, in request
    prep = self.prepare_request(req)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/sessions.py", line 436, in prepare_request
    hooks=merge_hooks(request.hooks, self.hooks),
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/models.py", line 302, in prepare
    self.prepare_url(url, params)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/models.py", line 382, in prepare_url
    raise MissingSchema(error)
requests.exceptions.MissingSchema: Invalid URL 'search_metadata': No schema supplied. Perhaps you mean

Exception in thread f9762a0b-37c7-4fa6-be17-d756c627258d:
Traceback (most recent call last):
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests_toolbelt/threaded/thread.py", line 30, in _handle_request
    response = self._session.request(**kwargs)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/sessions.py", line 504, in request
    prep = self.prepare_request(req)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/sessions.py", line 436, in prepare_request
    hooks=merge_hooks(request.hooks, self.hooks),
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/models.py", line 302, in prepare
    self.prepare_url(url, params)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/models.py", line 382, in prepare_url
    raise MissingSchema(error)
requests.exceptions.MissingSchema: Invalid URL 'statuses': No schema supplied. Perhaps you meant http:

Exception in thread 551abed8-949d-494f-85ad-b8ccc3012ba4:
Traceback (most recent call last):
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests_toolbelt/threaded/thread.py", line 30, in _handle_request
    response = self._session.request(**kwargs)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/sessions.py", line 504, in request
    prep = self.prepare_request(req)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/sessions.py", line 436, in prepare_request
    hooks=merge_hooks(request.hooks, self.hooks),
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/models.py", line 302, in prepare
    self.prepare_url(url, params)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/models.py", line 382, in prepare_url
    raise MissingSchema(error)
requests.exceptions.MissingSchema: Invalid URL 'statuses': No schema supplied. Perhaps you meant http:

Exception in thread 0a6cecfd-2978-411d-9204-697ced0248cf:
Traceback (most recent call last):
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests_toolbelt/threaded/thread.py", line 30, in _handle_request
    response = self._session.request(**kwargs)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/sessions.py", line 504, in request
    prep = self.prepare_request(req)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/sessions.py", line 436, in prepare_request
    hooks=merge_hooks(request.hooks, self.hooks),
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/models.py", line 302, in prepare
    self.prepare_url(url, params)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/models.py", line 382, in prepare_url
    raise MissingSchema(error)
requests.exceptions.MissingSchema: Invalid URL 'statuses': No schema supplied. Perhaps you meant http:

Exception in thread 51d292c6-4cf9-4f00-baf6-899b85953c41:
Traceback (most recent call last):
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests_toolbelt/threaded/thread.py", line 30, in _handle_request
    response = self._session.request(**kwargs)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/sessions.py", line 504, in request
    prep = self.prepare_request(req)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/sessions.py", line 436, in prepare_request
    hooks=merge_hooks(request.hooks, self.hooks),
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/models.py", line 302, in prepare
    self.prepare_url(url, params)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/models.py", line 382, in prepare_url
    raise MissingSchema(error)
requests.exceptions.MissingSchema: Invalid URL 'statuses': No schema supplied. Perhaps you meant http:

Exception in thread 9ad67967-89b6-4cb1-9b2d-3c4d874747db:
Traceback (most recent call last):
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests_toolbelt/threaded/thread.py", line 30, in _handle_request
    response = self._session.request(**kwargs)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/sessions.py", line 504, in request
    prep = self.prepare_request(req)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/sessions.py", line 436, in prepare_request
    hooks=merge_hooks(request.hooks, self.hooks),
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/models.py", line 302, in prepare
    self.prepare_url(url, params)
  File "/Users/Me/anaconda/envs/PythonData/lib/python3.6/site-packages/requests/models.py", line 382, in prepare_url
    raise MissingSchema(error)
requests.exceptions.MissingSchema: Invalid URL 'statuses': No schema supplied. Perhaps you meant http:

In [18]:
# We're going to store the tweets inside a dataframe associated with the state, location, and in-belt/out-belt 

In [19]:
# We're going to process all the tweets

In [20]:
# We're going to group the data in-belt/out-belt